In [3]:
import sys                                   # Pro pripojeni knihovny Open AI Gym
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihovny Open AI Gym
import gym
import pylab
import random
import numpy as np
from collections import deque
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential

EPISODES = 4000


class DQNAgent:
    def __init__(self, state_size, action_size):
        # Cartpole이 학습하는 것을 보려면 "True"로 바꿀 것
        self.render = True

        # state와 action의 크기를 가져와서 모델을 생성하는데 사용함
        self.state_size = state_size
        self.action_size = action_size

        # Cartpole DQN 학습의 Hyper parameter 들
        # deque를 통해서 replay memory 생성
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.005
        self.epsilon_decay = (self.epsilon - self.epsilon_min) / 50000
        self.batch_size = 64
        self.train_start = 1000
        self.memory = deque(maxlen=10000)

        # 학습할 모델과 타겟 모델을 생성
        self.model = self.build_model()
        self.target_model = self.build_model()
        # 학습할 모델을 타겟 모델로 복사 --> 타겟 모델의 초기화(weight를 같게 해주고 시작해야 함)
        self.update_target_model()

    # Deep Neural Network를 통해서 Q Function을 근사
    # state가 입력, 각 행동에 대한 Q Value가 출력인 모델을 생성
    def build_model(self):
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(self.action_size, activation='linear', kernel_initializer='he_uniform'))
        model.summary()
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    # 일정한 시간 간격마다 타겟 모델을 현재 학습하고 있는 모델로 업데이트
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    # 행동의 선택은 현재 네트워크에 대해서 epsilon-greedy 정책을 사용
    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q_value = self.model.predict(state)
            return np.argmax(q_value[0])

    # <s,a,r,s'>을 replay_memory에 저장함
    def replay_memory(self, state, action, reward, next_state, done):
        if action == 2:
            action = 1
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
        # print(len(self.memory))

    # replay memory에서 batch_size 만큼의 샘플들을 무작위로 뽑아서 학습
    def train_replay(self):
        if len(self.memory) < self.train_start:
            return
        batch_size = min(self.batch_size, len(self.memory))
        mini_batch = random.sample(self.memory, batch_size)

        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.action_size))

        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]

            # 큐러닝에서와 같이 s'에서의 최대 Q Value를 가져옴. 단, 타겟 모델에서 가져옴
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.discount_factor * \
                                          np.amax(self.target_model.predict(next_state)[0])
            update_input[i] = state
            update_target[i] = target

        # 학습할 정답인 타겟과 현재 자신의 값의 minibatch를 만들고 그것으로 한 번에 모델 업데이트
        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)

    # 저장한 모델을 불러옴
    def load_model(self, name):
        self.model.load_weights(name)

    # 학습된 모델을 저장함
    def save_model(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    # CartPole-v1의 경우 500 타임스텝까지 플레이가능
    env = gym.make('MountainCar-v0')
    # 환경으로부터 상태와 행동의 크기를 가져옴
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    #,action_size = 2
    # DQN 에이전트의 생성
    agent = DQNAgent(state_size, action_size)
#    agent.load_model("./save_model/MountainCar_DQN.h5")
    scores, episodes = [], []

    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        print(state)


        # 같은 액션을 4번하기 위한 카운터
        action_count = 0

        while not done:
            #if agent.render:
                #env.render()

            # 현재 상태에서 행동을 선택하고 한 스텝을 진행
            action_count = action_count + 1
            action = 0
            if action_count == 4:
                action = agent.get_action(state)
                action_count = 0


            # 선택한 액션으로 1 step을 시행한다
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            # 에피소드를 끝나게 한 행동에 대해서 -100의 패널티를 줌
            #reward = reward if not done else -100

            # <s, a, r, s'>을 replay memory에 저장
            agent.replay_memory(state, action, reward, next_state, done)
            # 매 타임스텝마다 학습을 진행
            agent.train_replay()
            score += reward
            state = next_state

            if done:
                env.reset()
                # 매 에피소드마다 학습하는 모델을 타겟 모델로 복사
                agent.update_target_model()

                # 각 에피소드마다 cartpole이 서있었던 타임스텝을 plot
                scores.append(score)
                episodes.append(e)
                #pylab.plot(episodes, scores, 'b')
                #pylab.savefig("./save_graph/MountainCar_DQN.png")
                print("episode:", e, "  score:", score, "  memory length:", len(agent.memory),
                      "  epsilon:", agent.epsilon)

        # 50 에피소드마다 학습 모델을 저장
        if e % 50 == 0:
             agent.save_model("./MountainCar_DQN.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 32)                96        
_________________________________________________________________
dense_14 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 51        
Total params: 675
Trainable params: 675
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                96        
_________________________________________________________________
dense_17 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_18 (De

('episode:', 54, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7811000000004222)
[[-0.43922765  0.        ]]
('episode:', 55, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7771200000004299)
[[-0.55269596  0.        ]]
('episode:', 56, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7731400000004376)
[[-0.53781116  0.        ]]
('episode:', 57, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7691600000004453)
[[-0.5946463  0.       ]]
('episode:', 58, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7651800000004529)
[[-0.55456539  0.        ]]
('episode:', 59, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7612000000004606)
[[-0.59924938  0.        ]]
('episode:', 60, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.7572200000004683)
[[-0.45547354  0.        ]]
('episode:', 61, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.753240000000476)
[[-0.50773979  0.      

('episode:', 120, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.5184200000009289)
[[-0.4698853  0.       ]]
('episode:', 121, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.5144400000009366)
[[-0.50313068  0.        ]]
('episode:', 122, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.5104600000009443)
[[-0.59886475  0.        ]]
('episode:', 123, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.506480000000952)
[[-0.59699016  0.        ]]
('episode:', 124, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.5025000000009596)
[[-0.42919867  0.        ]]
('episode:', 125, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.49852000000096314)
[[-0.52821618  0.        ]]
('episode:', 126, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.4945400000009597)
[[-0.50870463  0.        ]]
('episode:', 127, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.4905600000009563)
[[-0.43318162 

('episode:', 185, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.2597200000007576)
[[-0.5195224  0.       ]]
('episode:', 186, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.2557400000007542)
[[-0.52480347  0.        ]]
('episode:', 187, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.25176000000075077)
[[-0.52663866  0.        ]]
('episode:', 188, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.24778000000075046)
[[-0.48284631  0.        ]]
('episode:', 189, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.24380000000075258)
[[-0.4299027  0.       ]]
('episode:', 190, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.2398200000007547)
[[-0.43193227  0.        ]]
('episode:', 191, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.23584000000075683)
[[-0.54197032  0.        ]]
('episode:', 192, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.23186000000075896)
[[-0.598141

('episode:', 250, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.43372142  0.        ]]
('episode:', 251, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.5093898  0.       ]]
('episode:', 252, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.48426416  0.        ]]
('episode:', 253, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.40536715  0.        ]]
('episode:', 254, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.58150223  0.        ]]
('episode:', 255, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.43670826  0.        ]]
('episode:', 256, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.54615189  0.        ]]
('episode:', 257, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017

...

('episode:', 1978, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.5599163  0.       ]]
('episode:', 1979, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.45642703  0.        ]]
('episode:', 1980, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.46113595  0.        ]]
('episode:', 1981, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.46104684  0.        ]]
('episode:', 1982, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.51674526  0.        ]]
('episode:', 1983, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.55670941  0.        ]]
('episode:', 1984, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.59720391  0.        ]]
('episode:', 1985, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.0049801000

('episode:', 2042, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.56387142  0.        ]]
('episode:', 2043, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.43052854  0.        ]]
('episode:', 2044, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.55411777  0.        ]]
('episode:', 2045, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.44434892  0.        ]]
('episode:', 2046, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.52283403  0.        ]]
('episode:', 2047, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.50060894  0.        ]]
('episode:', 2048, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.57003476  0.        ]]
('episode:', 2049, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.00498010

('episode:', 2106, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.41119132  0.        ]]
('episode:', 2107, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.55581986  0.        ]]
('episode:', 2108, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.49815902  0.        ]]
('episode:', 2109, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.48817023  0.        ]]
('episode:', 2110, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.51249941  0.        ]]
('episode:', 2111, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.42507714  0.        ]]
('episode:', 2112, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.52332456  0.        ]]
('episode:', 2113, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.00498010

('episode:', 2170, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.45539717  0.        ]]
('episode:', 2171, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.52716587  0.        ]]
('episode:', 2172, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.51394077  0.        ]]
('episode:', 2173, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.53035005  0.        ]]
('episode:', 2174, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.44669981  0.        ]]
('episode:', 2175, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.43372036  0.        ]]
('episode:', 2176, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.004980100000801017)
[[-0.42530043  0.        ]]
('episode:', 2177, '  score:', -200.0, '  memory length:', 10000, '  epsilon:', 0.00498010

KeyboardInterrupt: 